In [2]:
import pandas as pd
import numpy as np
import nltk
import nltk.sentiment.util
from nltk.tokenize import sent_tokenize, word_tokenize
import matplotlib.pyplot as plt
import pprint
import re
import gensim
import gensim.corpora as corpora
import os
from gensim.utils import simple_preprocess
from nltk.corpus import stopwords
import string
import math

In [3]:
from nltk.stem.porter import PorterStemmer
stemmer = PorterStemmer()

In [4]:
df = pd.read_csv('../../amazon_reviews_us_Grocery_v1_00_sentiment.csv').sample(100000)

df = df.drop(["marketplace", "customer_id", "product_parent", "product_category", "review_date"], axis=1)

df = df[~df['review_body'].isnull()]

df['review_body'] = df['review_headline'] + '. ' + df['review_body']

df["health_hazard"] = np.nan

df = df.set_index("review_id")

In [5]:
df

product_id                                      product_title  \
review_id                                                                       
RPT3T1P1JHR2E   B00O85SOK6  Gimme All Natural Roasted Seaweed Crumbles Ses...   
R11HVUX8YW5IY4  B00EZC766A  South Beach Diet Snack Bar, Fudgy Chocolate Mi...   
R373ISBSUPKCC8  B0049Z7Q8Q  Rodelle Gourmet Baking Cocoa, 8-Ounce (Pack of 2)   
R30HKNGSPEGV4P  B001IZHU54  Breath Savers 3-Hour Mints 1.27-Ounce (Pack of...   
R2XZDJDC1YSDRT  B00BN1U8WK              Sprout Fruity Yogurt Bites, 1.1 Ounce   
...                    ...                                                ...   
R3BMYY2H6G7Y2T  B004G90TNO                           Maltesers Large Bag 135g   
R6Y4SR5QKN2TU   B0055IRNAC        Anthony's Blanched Gluten Free Almond Flour   
R125SWAXV682LH  B005GNXG1G  Godiva French Vanilla Coffee, 12 Ounce Bag (Pa...   
RD4PVGBH7FYEB   B0039VRNUW                             Pumpkin Roll - 8x8 Pan   
R32479SP48DZP8  B000CHIJ0Y                             Ghirardelli Sauce Pump   

                star_rating  helpful_votes  total_votes vine  \
review_id                                                      
RPT3T1P1JHR2E             5              1            1    N   
R11HVUX8YW5IY4            5              0            0    N   
R373ISBSUPKCC8            5              0            0    N   
R30HKNGSPEGV4P            5              0            0    N   
R2XZDJDC1YSDRT            5              0            0    N   
...                     ...            ...          ...  ...   
R3BMYY2H6G7Y2T            4              0            0    N   
R6Y4SR5QKN2TU             5              0            0    N   
R125SWAXV682LH            5              0            0    N   
RD4PVGBH7FYEB             5              1            1    N   
R32479SP48DZP8            3              0            0    N   

               verified_purchase  \
review_id                          
RPT3T1P1JHR2E                  N   
R11HVUX8YW5IY4                 Y   
R373ISBSUPKCC8                 N   
R30HKNGSPEGV4P                 N   
R2XZDJDC1YSDRT                 Y   
...                          ...   
R3BMYY2H6G7Y2T                 N   
R6Y4SR5QKN2TU                  Y   
R125SWAXV682LH                 Y   
RD4PVGBH7FYEB                  Y   
R32479SP48DZP8                 Y   

                                                  review_headline  \
review_id                                                           
RPT3T1P1JHR2E                 It's a fabulous healthy alternative   
R11HVUX8YW5IY4                         Delicious and Recommended!   
R373ISBSUPKCC8                                    Seriously Yummy   
R30HKNGSPEGV4P                          Best mint I have ever had   
R2XZDJDC1YSDRT                            Toddler chooses Sprouts   
...                                                           ...   
R3BMYY2H6G7Y2T  A little too sweet but better than Whoppers. M...   
R6Y4SR5QKN2TU              Terrific price for great quality flour   
R125SWAXV682LH                                         Five Stars   
RD4PVGBH7FYEB                                 These are wonderful   
R32479SP48DZP8                           Just another item to buy   

                                                      review_body  polarity  \
review_id                                                                     
RPT3T1P1JHR2E   It's a fabulous healthy alternative. Healthy, ...  0.282066   
R11HVUX8YW5IY4             Delicious and Recommended!. Delicious!  1.000000   
R373ISBSUPKCC8  Seriously Yummy. Rodelle's Cocoa is completely...  0.033929   
R30HKNGSPEGV4P  Best mint I have ever had. Best mint I have ev...  0.500000   
R2XZDJDC1YSDRT  Toddler chooses Sprouts. My 2 year old loves t...  0.293750   
...                                                           ...       ...   
R3BMYY2H6G7Y2T  A little too sweet but better than Whoppers. M...  0.115466   
R6Y4SR5QKN2TU   Terrific price for great quality fl

In [6]:
neg_reviews = df.query("star_rating < 4", engine="python")

In [7]:
stop_words = stopwords.words('english')
stop_words.extend(['br'])
translator=str.maketrans('','',string.punctuation)

#neg_reviews["tokened_review"] = np.nan
all_tokens = []
#neg_reviews.astype({"tokened_review": 'object'}).dtypes
for index, row in neg_reviews.iterrows():
    text = row["review_body"].lower()
    text = text.translate(translator)
    text = word_tokenize(text)
    new_text = []
    for token in text:
        token = stemmer.stem(token)
        if token not in stop_words:
            new_text.append(token)
    all_tokens.append(new_text)
neg_reviews["tokened_review"] = all_tokens

<ipython-input-7-eba5c76ea0a6>:18: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  neg_reviews["tokened_review"] = all_tokens


In [8]:
neg_reviews

product_id                                      product_title  \
review_id                                                                       
R1IX3K6G5YPQ1B  B000VK3RPY                                        Baking Soda   
R2K2JBHPYSZ6TH  B0084Y9CLC  Sincerely Nuts Chocolate Rocks Candy Nuggets 1...   
R77UG2XCZ94C1   B002T5IPJC                        48oz. Light Fruit Cake Ring   
R1ID2E4X8BHCMB  B00DYW1ZTQ  Happy Baby Organic Baby Food Stage 3 Hearty Me...   
R2JLI0T7RHW8HM  B0014RK2F6  Stevita Stevia Sweetened Drink Mix, Cherry, 2....   
...                    ...                                                ...   
R3MITBTVKDF9GP  B00IJNAOE2                    M&M's Mega Peanut 11.40 Oz. Bag   
R224THFUW5F3BY  B000DZKKMU    Pamela's Products Chunky Chocolate Chip Cookies   
R1RGSI4WUW80SA  B000H7KAME          India Tree Light Muscovado Sugar, 1 Pound   
R3FX52J4XQ438G  B000FMMTHK            Waggin Train Chicken Jerky Tenders 40oz   
R32479SP48DZP8  B000CHIJ0Y                             Ghirardelli Sauce Pump   

                star_rating  helpful_votes  total_votes vine  \
review_id                                                      
R1IX3K6G5YPQ1B            1              9           21    N   
R2K2JBHPYSZ6TH            1              0            0    N   
R77UG2XCZ94C1             3              0            0    N   
R1ID2E4X8BHCMB            1             10           13    N   
R2JLI0T7RHW8HM            3              5            6    N   
...                     ...            ...          ...  ...   
R3MITBTVKDF9GP            2              1            5    N   
R224THFUW5F3BY            3              0            0    N   
R1RGSI4WUW80SA            3              3            4    N   
R3FX52J4XQ438G            1              0            0    N   
R32479SP48DZP8            3              0            0    N   

               verified_purchase  \
review_id                          
R1IX3K6G5YPQ1B                 N   
R2K2JBHPYSZ6TH                 Y   
R77UG2XCZ94C1                  Y   
R1ID2E4X8BHCMB                 Y   
R2JLI0T7RHW8HM                 Y   
...                          ...   
R3MITBTVKDF9GP                 N   
R224THFUW5F3BY                 Y   
R1RGSI4WUW80SA                 Y   
R3FX52J4XQ438G                 N   
R32479SP48DZP8                 Y   

                                                  review_headline  \
review_id                                                           
R1IX3K6G5YPQ1B         Low rating because of marketing deception.   
R2K2JBHPYSZ6TH                                   Bait and switch!   
R77UG2XCZ94C1                                         Three Stars   
R1ID2E4X8BHCMB  I happily ordered the "Happy Baby Organic Baby...   
R2JLI0T7RHW8HM                        Hey, this is pretty good !!   
...                                                           ...   
R3MITBTVKDF9GP                       Bigger is not always better!   
R224THFUW5F3BY                                        Three Stars   
R1RGSI4WUW80SA  It has a good flavor but the price is too high...   
R3FX52J4XQ438G                                            Deadly?   
R32479SP48DZP8                           Just another item to buy   

                                                      review_body  polarity  \
review_id                                                                     
R1IX3K6G5YPQ1B  Low rating because of marketing deception.. NO...  0.033333   
R2K2JBHPYSZ6TH  Bait and switch!. I bought these because they ... -0.113690   
R77UG2XCZ94C1               Three Stars. Great quality and taste!  1.000000   
R1ID2E4X8BHCMB  I happily ordered the "Happy Baby Organic Baby...  0.457407   
R2JLI0T7RHW8HM  Hey, this is pretty good !!. The first thing t...  0.061060   
...                                                           ...       ...   
R3MITBTVKDF9GP  Bigger is not always better!. Now, don't get m... -0.113021   
R224THFUW5F3BY      Three Stars. really not my favo

In [9]:
hazardous_words=["rotten", "mouldy", "moldy", "mold", "mould", "sick", "dangerous", "diarrhea", "poisoning", "stale", "contaminate",'disease', 'disgusting', 'hospital', 'poisoning', 'hazardous', 'ill', 'mold','moldy', 'mildew', 'dangerous' 'rancid', 'smelly', 'stale', 'unhealthy', 'sick','expired','expiration']
for word in hazardous_words:
    neg_reviews.loc[[stemmer.stem(word) in tokened_review for tokened_review in neg_reviews["tokened_review"]], "health_hazard"] = 1
    

C:\ProgramData\Anaconda3\lib\site-packages\pandas\core\indexing.py:1765: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  isetter(loc, value)


In [10]:
num_labeled = len(neg_reviews.query("health_hazard == 1", engine="python"))
num_labeled

2077

In [11]:
non_hazardous = neg_reviews.query("health_hazard != 1", engine="python").sample(3*num_labeled)
for i in list(non_hazardous.index):
    neg_reviews.loc[i, "health_hazard"] = 0

C:\ProgramData\Anaconda3\lib\site-packages\pandas\core\indexing.py:1765: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  isetter(loc, value)


In [12]:
labeled = neg_reviews.query("health_hazard == 0 or health_hazard == 1", engine="python")
reviews = labeled.loc[:, 'tokened_review'].values
y = labeled.loc[:, 'health_hazard'].values

In [13]:
pprint.pprint(reviews)

array([list(['hey', 'thi', 'pretti', 'good', 'first', 'thing', 'notic', 'jar', 'littl', '13', 'serv', 'per', 'jar', 'fragranc', 'like', 'good', 'old', 'cherri', 'kid', 'drink', 'flavor', 'pretti', 'good', 'stevia', 'product', 'aftertast', 'imo', 'thi', 'pretti', 'good', 'drink', 'lot', 'ice', 'help', 'im', 'sure', 'id', 'drink', 'daili', 'nice', 'repriev', 'plain', 'water', 'cant', 'drink', 'artifici', 'sweeten', 'sugarfre', 'drink', 'nice', 'like', 'truelemon', 'product', 'better', 'plain', 'cherri', 'flavor', 'thi', 'okbr', 'edit', '7222012br', 'aftertast', 'significantprob', 'reorder']),
       list(['good', 'product', 'ive', 'tri', 'peach', 'green', 'tea', 'steaz', 'thought', 'thi', 'would', 'tast', 'similar', 'think', 'someth', 'lose', 'translat', 'far', 'thi', 'product', 'zero', 'calori', 'sweet', 'fullflavor', 'love', 'brand', 'thi', 'drink']),
       list(['didnt', 'order', 'decaf', 'receiv', '6', 'decaf', 'k', 'cup', 'dont', 'drink', 'decaf', 'wa', 'wast', 'degre', 'howev', 'r

In [14]:
for i in range(len(reviews)):
    reviews[i] = ' '.join(reviews[i])

In [15]:
reviews

array(['hey thi pretti good first thing notic jar littl 13 serv per jar fragranc like good old cherri kid drink flavor pretti good stevia product aftertast imo thi pretti good drink lot ice help im sure id drink daili nice repriev plain water cant drink artifici sweeten sugarfre drink nice like truelemon product better plain cherri flavor thi okbr edit 7222012br aftertast significantprob reorder',
       'good product ive tri peach green tea steaz thought thi would tast similar think someth lose translat far thi product zero calori sweet fullflavor love brand thi drink',
       'didnt order decaf receiv 6 decaf k cup dont drink decaf wa wast degre howev rest coffe wa great know decaf wa packag would purchas someth els',
       ...,
       'stay away buyer bewar piec crap full rock sand bag insid kcup send kcup brown bag stapl like wtf quit danger ask never would tast one would ever buy form thi compani know could poison',
       'ha good flavor price high ha good flavor price high onli

In [16]:
from sklearn.feature_extraction.text import TfidfVectorizer
matrix = TfidfVectorizer(max_features=200)
X = matrix.fit_transform(reviews).toarray()
X

array([[0.        , 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ],
       [0.        , 0.        , 0.        , ..., 0.        , 0.18095889,
        0.        ],
       [0.        , 0.        , 0.        , ..., 0.        , 0.18226248,
        0.        ],
       ...,
       [0.        , 0.        , 0.        , ..., 0.        , 0.29806155,
        0.        ],
       [0.        , 0.        , 0.        , ..., 0.        , 0.24346232,
        0.        ],
       [0.        , 0.        , 0.        , ..., 0.        , 0.        ,
        0.49178796]])

In [17]:
# split train and test data
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y)

In [18]:
from sklearn.naive_bayes import GaussianNB
classifier_g = GaussianNB()

from sklearn.linear_model import LogisticRegression

classifier_l = LogisticRegression()

classifier_g.fit(X_train, y_train)
classifier_l.fit(X_train, y_train)

# Predict Class
y_pred_g = classifier_g.predict(X_test)
y_pred_l = classifier_l.predict(X_test)

# Accuracy 
from sklearn.metrics import accuracy_score
accuracy_g = accuracy_score(y_test, y_pred_g)
accuracy_l = accuracy_score(y_test, y_pred_l)
print(accuracy_g)

0.9427058257101589


In [19]:
trial = neg_reviews.sample(100)

In [20]:
matrix.get_feature_names()

['actual',
 'ad',
 'add',
 'almost',
 'also',
 'amazon',
 'ani',
 'anoth',
 'anyth',
 'arriv',
 'away',
 'back',
 'bad',
 'bag',
 'bar',
 'bean',
 'becaus',
 'befor',
 'best',
 'better',
 'big',
 'bit',
 'bitter',
 'bottl',
 'bought',
 'box',
 'brand',
 'buy',
 'came',
 'candi',
 'cant',
 'chang',
 'chip',
 'chocol',
 'coffe',
 'come',
 'compani',
 'contain',
 'cooki',
 'could',
 'cup',
 'date',
 'day',
 'didnt',
 'differ',
 'disappoint',
 'disgust',
 'doe',
 'doesnt',
 'dont',
 'dri',
 'drink',
 'eat',
 'end',
 'enjoy',
 'enough',
 'even',
 'ever',
 'expect',
 'expens',
 'expir',
 'feel',
 'find',
 'fine',
 'first',
 'flavor',
 'food',
 'found',
 'free',
 'fresh',
 'get',
 'give',
 'go',
 'good',
 'got',
 'great',
 'green',
 'ha',
 'half',
 'hard',
 'high',
 'hope',
 'horribl',
 'hot',
 'howev',
 'id',
 'ill',
 'im',
 'ingredi',
 'item',
 'ive',
 'kcup',
 'kind',
 'know',
 'label',
 'last',
 'less',
 'like',
 'littl',
 'local',
 'look',
 'lot',
 'love',
 'made',
 'make',
 'mani',
 'ma

In [21]:
trial = neg_reviews
trial_x = trial.loc[:,"tokened_review"].values
for i in range(len(trial_x)):
    trial_x[i] = ' '.join(trial_x[i])
trial_X = matrix.fit_transform(trial_x).toarray()
trial_y = classifier_l.predict(trial_X)
trial_y

array([0., 0., 0., ..., 0., 1., 0.])

In [22]:
neg_reviews["health_hazard"] = trial_y

<ipython-input-22-56d181f868ab>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  neg_reviews["health_hazard"] = trial_y


In [23]:
neg_reviews

product_id                                      product_title  \
review_id                                                                       
R1IX3K6G5YPQ1B  B000VK3RPY                                        Baking Soda   
R2K2JBHPYSZ6TH  B0084Y9CLC  Sincerely Nuts Chocolate Rocks Candy Nuggets 1...   
R77UG2XCZ94C1   B002T5IPJC                        48oz. Light Fruit Cake Ring   
R1ID2E4X8BHCMB  B00DYW1ZTQ  Happy Baby Organic Baby Food Stage 3 Hearty Me...   
R2JLI0T7RHW8HM  B0014RK2F6  Stevita Stevia Sweetened Drink Mix, Cherry, 2....   
...                    ...                                                ...   
R3MITBTVKDF9GP  B00IJNAOE2                    M&M's Mega Peanut 11.40 Oz. Bag   
R224THFUW5F3BY  B000DZKKMU    Pamela's Products Chunky Chocolate Chip Cookies   
R1RGSI4WUW80SA  B000H7KAME          India Tree Light Muscovado Sugar, 1 Pound   
R3FX52J4XQ438G  B000FMMTHK            Waggin Train Chicken Jerky Tenders 40oz   
R32479SP48DZP8  B000CHIJ0Y                             Ghirardelli Sauce Pump   

                star_rating  helpful_votes  total_votes vine  \
review_id                                                      
R1IX3K6G5YPQ1B            1              9           21    N   
R2K2JBHPYSZ6TH            1              0            0    N   
R77UG2XCZ94C1             3              0            0    N   
R1ID2E4X8BHCMB            1             10           13    N   
R2JLI0T7RHW8HM            3              5            6    N   
...                     ...            ...          ...  ...   
R3MITBTVKDF9GP            2              1            5    N   
R224THFUW5F3BY            3              0            0    N   
R1RGSI4WUW80SA            3              3            4    N   
R3FX52J4XQ438G            1              0            0    N   
R32479SP48DZP8            3              0            0    N   

               verified_purchase  \
review_id                          
R1IX3K6G5YPQ1B                 N   
R2K2JBHPYSZ6TH                 Y   
R77UG2XCZ94C1                  Y   
R1ID2E4X8BHCMB                 Y   
R2JLI0T7RHW8HM                 Y   
...                          ...   
R3MITBTVKDF9GP                 N   
R224THFUW5F3BY                 Y   
R1RGSI4WUW80SA                 Y   
R3FX52J4XQ438G                 N   
R32479SP48DZP8                 Y   

                                                  review_headline  \
review_id                                                           
R1IX3K6G5YPQ1B         Low rating because of marketing deception.   
R2K2JBHPYSZ6TH                                   Bait and switch!   
R77UG2XCZ94C1                                         Three Stars   
R1ID2E4X8BHCMB  I happily ordered the "Happy Baby Organic Baby...   
R2JLI0T7RHW8HM                        Hey, this is pretty good !!   
...                                                           ...   
R3MITBTVKDF9GP                       Bigger is not always better!   
R224THFUW5F3BY                                        Three Stars   
R1RGSI4WUW80SA  It has a good flavor but the price is too high...   
R3FX52J4XQ438G                                            Deadly?   
R32479SP48DZP8                           Just another item to buy   

                                                      review_body  polarity  \
review_id                                                                     
R1IX3K6G5YPQ1B  Low rating because of marketing deception.. NO...  0.033333   
R2K2JBHPYSZ6TH  Bait and switch!. I bought these because they ... -0.113690   
R77UG2XCZ94C1               Three Stars. Great quality and taste!  1.000000   
R1ID2E4X8BHCMB  I happily ordered the "Happy Baby Organic Baby...  0.457407   
R2JLI0T7RHW8HM  Hey, this is pretty good !!. The first thing t...  0.061060   
...                                                           ...       ...   
R3MITBTVKDF9GP  Bigger is not always better!. Now, don't get m... -0.113021   
R224THFUW5F3BY      Three Stars. really not my favo

In [24]:
dangerous = neg_reviews.query("health_hazard == 1", engine="python")
dangerous

product_id                                      product_title  \
review_id                                                                       
R3AGZHZ97HVV2V  B004ED95GY     Steaz Organic Iced Teaz, 16 Ounce (Pack of 12)   
R3G0TRZUXU3OR4  B000VK88IA                            Enjoy Life Not Nuts Mix   
RVMQDTNJF5TQR   B007UXVPBS  15 Deep Dark Roast Sampler Pack, You Are Guara...   
R25MJIG1Z01SGR  B00ARZB22S  Wonka Sweetarts Valentine Candy & Card Kit - V...   
R33LCBMFUWQO9J  B004F1LNDI  Honey Stinger Organic Waffle, Honey, 1.06 Ounc...   
...                    ...                                                ...   
R2PAJ4TULMTFWN  B00J074W7Q  Orgain Organic Plant Based Protein Powder, Pac...   
R14C9DNZ3NV4Z5  B00KSF4LZM  Angelino's Coffee Variety Pack for Keurig K-cu...   
R38RPGTPFWSHMW  B001ONTM38  Special K Bliss Bars, Raspberry, 5-Count Bars ...   
R3721QGZLV3TUC  B000CN7BMA  ZICO Natural Coconut Water,11.2 fl. oz. (Pack ...   
R3FX52J4XQ438G  B000FMMTHK            Waggin Train Chicken Jerky Tenders 40oz   

                star_rating  helpful_votes  total_votes vine  \
review_id                                                      
R3AGZHZ97HVV2V            3              0            0    N   
R3G0TRZUXU3OR4            2              0            2    N   
RVMQDTNJF5TQR             3              0            0    N   
R25MJIG1Z01SGR            1              0            0    N   
R33LCBMFUWQO9J            3              0            1    N   
...                     ...            ...          ...  ...   
R2PAJ4TULMTFWN            3              1            2    N   
R14C9DNZ3NV4Z5            1              0            3    N   
R38RPGTPFWSHMW            3              0            0    N   
R3721QGZLV3TUC            1              0            0    N   
R3FX52J4XQ438G            1              0            0    N   

               verified_purchase  \
review_id                          
R3AGZHZ97HVV2V                 Y   
R3G0TRZUXU3OR4                 N   
RVMQDTNJF5TQR                  Y   
R25MJIG1Z01SGR                 Y   
R33LCBMFUWQO9J                 Y   
...                          ...   
R2PAJ4TULMTFWN                 N   
R14C9DNZ3NV4Z5                 Y   
R38RPGTPFWSHMW                 Y   
R3721QGZLV3TUC                 Y   
R3FX52J4XQ438G                 N   

                                                  review_headline  \
review_id                                                           
R3AGZHZ97HVV2V                Not as good as their other products   
R3G0TRZUXU3OR4                                     Not very tasty   
RVMQDTNJF5TQR                                  Didn't order decaf   
R25MJIG1Z01SGR                                      Too expensive   
R33LCBMFUWQO9J  As energy foods go, these taste pretty good. H...   
...                                                           ...   
R2PAJ4TULMTFWN                                         not for me   
R14C9DNZ3NV4Z5  Product is NOT compatible with keurig machine....   
R38RPGTPFWSHMW  I love these bars, but they taste stale from t...   
R3721QGZLV3TUC                      FROM the STARS to the STAHLS!   
R3FX52J4XQ438G                                            Deadly?   

                                                      review_body  polarity  \
review_id                                                                     
R3AGZHZ97HVV2V  Not as good as their other products. I've trie...  0.272222   
R3G0TRZUXU3OR4  Not very tasty. Was hoping, given the reviews,... -0.096875   
RVMQDTNJF5TQR   Didn't order decaf. I received 6 decaf K Cups....  0.150000   
R25MJIG1Z01SGR  Too expensive. I guess I was not really paying... -0.122222   
R33LCBMFUWQO9J  As energy foods go, these taste pretty good. H...  0.143229   
...                                                           ...       ...   
R2PAJ4TULMTFWN  not for me. So recently my boyfriend and best ...  0.073671   
R14C9DNZ3NV4Z5  Product is NOT compatible with keur

In [25]:
#LDA
data_filtered = dangerous['review_body'].values
pprint.pprint(data_filtered)

array(["Not as good as their other products. I've tried the Peach green tea by Steaz and thought this would taste similar, but I think something is lose in translation as far as how this product is zero calories. Not as sweet or full-flavored. Love the brand, just not this drink.",
       'Not very tasty. Was hoping, given the reviews, for something a little tastier.  Very fruit & seed heavy and tastes of it.  Imagine dried cranberries and sunflower seeds.  My sister has nut allergies, so I was looking for something she would like, but I think I am going to have to keep looking',
       "Didn't order decaf. I received 6 decaf K Cups. I don't drink decaf so it was a waste to a degree. However the rest if the coffee was great. If I had know decaf was in the package I would have purchased something else.",
       ...,
       "I love these bars, but they taste stale from this supplier. First of all, I'd like to begin by saying that I love anything that has chocolate and raspberry flavoring

In [26]:
def sent_to_words(sentences):
    for sentence in sentences:
        sentence = sentence.lower()
        sentence = sentence.translate(translator)
        yield(nltk.word_tokenize(str(sentence)))

data_words = list(sent_to_words(data_filtered))
clean_data = []
for i in range(len(data_words)):
    clean_review = []
    for j in range(len(data_words[i])):
        if data_words[i][j] not in stop_words:
            clean_review.append(data_words[i][j])
    clean_data.append(clean_review)

In [27]:
# Create Dictionary
id2word = corpora.Dictionary(clean_data)
# Create Corpus
texts = clean_data
# Term Document Frequency
corpus = [id2word.doc2bow(text) for text in texts]
# View
pprint.pprint(corpus[:5])

[[(0, 1),
  (1, 1),
  (2, 1),
  (3, 1),
  (4, 1),
  (5, 1),
  (6, 1),
  (7, 1),
  (8, 1),
  (9, 1),
  (10, 1),
  (11, 1),
  (12, 1),
  (13, 1),
  (14, 1),
  (15, 1),
  (16, 1),
  (17, 1),
  (18, 1),
  (19, 1),
  (20, 1),
  (21, 1),
  (22, 1),
  (23, 1),
  (24, 1)],
 [(14, 2),
  (19, 1),
  (23, 1),
  (25, 1),
  (26, 1),
  (27, 1),
  (28, 1),
  (29, 1),
  (30, 1),
  (31, 1),
  (32, 1),
  (33, 1),
  (34, 1),
  (35, 1),
  (36, 1),
  (37, 2),
  (38, 1),
  (39, 1),
  (40, 1),
  (41, 1),
  (42, 1),
  (43, 1),
  (44, 1),
  (45, 1),
  (46, 1)],
 [(2, 1),
  (14, 1),
  (23, 1),
  (47, 1),
  (48, 1),
  (49, 1),
  (50, 4),
  (51, 1),
  (52, 1),
  (53, 1),
  (54, 1),
  (55, 1),
  (56, 1),
  (57, 1),
  (58, 1),
  (59, 1),
  (60, 1),
  (61, 1),
  (62, 1),
  (63, 1),
  (64, 1)],
 [(14, 1),
  (20, 1),
  (23, 1),
  (65, 1),
  (66, 1),
  (67, 1),
  (68, 1),
  (69, 2),
  (70, 1),
  (71, 1),
  (72, 1),
  (73, 1),
  (74, 1),
  (75, 1)],
 [(2, 2),
  (5, 3),
  (17, 1),
  (30, 1),
  (34, 1),
  (35, 1),
  (52, 1

In [28]:
# number of topics
num_topics = 5
# Build LDA model
lda_model = gensim.models.LdaMulticore(corpus=corpus,
                                       id2word=id2word,
                                       num_topics=num_topics,
                                       minimum_probability=0.1,
                                       workers=3,
                                       passes=2)
# Print the Keyword in the 10 topics
pprint.pprint(lda_model.print_topics())
doc_lda = lda_model[corpus]

[(0,
  '0.011*"like" + 0.009*"coffee" + 0.008*"taste" + 0.007*"product" + '
  '0.006*"something" + 0.006*"one" + 0.006*"would" + 0.006*"good" + '
  '0.006*"different" + 0.005*"flavor"'),
 (1,
  '0.010*"product" + 0.007*"coffee" + 0.006*"something" + 0.006*"tea" + '
  '0.006*"flavor" + 0.005*"taste" + 0.005*"good" + 0.005*"one" + 0.005*"ill" + '
  '0.005*"get"'),
 (2,
  '0.009*"like" + 0.008*"taste" + 0.007*"product" + 0.007*"good" + 0.007*"one" '
  '+ 0.006*"different" + 0.006*"flavor" + 0.006*"something" + 0.005*"even" + '
  '0.004*"great"'),
 (3,
  '0.013*"like" + 0.010*"something" + 0.010*"taste" + 0.008*"good" + '
  '0.007*"one" + 0.007*"would" + 0.006*"different" + 0.006*"dont" + '
  '0.006*"coffee" + 0.006*"ill"'),
 (4,
  '0.014*"like" + 0.010*"taste" + 0.008*"something" + 0.008*"different" + '
  '0.007*"would" + 0.006*"tea" + 0.005*"one" + 0.005*"product" + 0.005*"good" '
  '+ 0.005*"sugar"')]


In [29]:
#from itertools import chain
#lda_corpus = lda_model[corpus]

# Find the threshold, let's set the threshold to be 1/#clusters,
# To prove that the threshold is sane, we average the sum of all probabilities:
#scores = list(chain(*[[score for topic_id,score in topic] for topic in [doc for doc in lda_corpus]]))
#threshold = sum(scores)/len(scores)


In [30]:
import pickle 
import pyLDAvis
import pyLDAvis.gensim_models
data_vis = pyLDAvis.gensim_models.prepare(lda_model, corpus, id2word)

pyLDAvis.display(data_vis)